In [ ]:
import os
import sys

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import shapely
from tqdm.notebook import tqdm

parent_dir = os.path.split(os.getcwd())[0]
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from scripts import dl_utils

In [ ]:
center_coord = (-56.71779286811243,-5.806900403219626)
num_pixels = 1000

START_DATE = '2021-01-01'
END_DATE = '2022-01-01'
METHOD = 'median'
MOSAIC_PERIOD = 4  # the period over which to mosaic image data in months
MAX_CLOUD = 0.25  # maximum cloud cover for a tile to be included in the dataset

# Convert pixels to degrees. Heuristic, not geographically sound
rect_width = np.round((num_pixels / 100) / 111.32, 4)    
polygons = [dl_utils.rect_from_point(center_coord, rect_width)]

patches = []
patch_dates = []
progress_counter = 0

In [ ]:
for polygon in tqdm(polygons[progress_counter:]):
    try:
        data = dl_utils.SentinelData(polygon, START_DATE, END_DATE, MOSAIC_PERIOD, method=METHOD)
        rect_width = rect_width
        data.search_scenes()
        if len(data.scenes) > 0:
            data.download_scenes()
            data.create_composites()
            composites = data.composites
            dates = data.composite_dates
            bounds = data.metadata[0]["wgs84Extent"]["coordinates"][0][:-1]
            data.compute_cloud_fraction()
            patches += [p for p, cloud in zip(composites, data.cloud_fraction) if cloud < MAX_CLOUD]
            patch_dates += [d for d, cloud in zip(dates, data.cloud_fraction) if cloud < MAX_CLOUD]
        else:
            print("No cloud-free scenes found")
    except KeyboardInterrupt:
        print("Keyboard Interrupt!")
        break
    except Exception as e:
        print('Failure', polygon)
        print(e)
    progress_counter += 1

In [ ]:
plt.figure(figsize=(len(composites) * 3, 3), dpi=250, facecolor=(1,1,1))
for i in range(len(composites)):
    plt.subplot(1,len(composites),i +1)
    plt.imshow(np.clip(composites[i][:,:,3:0:-1] / 3000, 0, 1))
    plt.title(f'{dates[i]}')
    plt.axis('off')
plt.suptitle(f'Composites - {center_coord[0]:.3f}, {center_coord[1]:.3f}')
plt.tight_layout()
plt.savefig(f'../figures/example images {num_pixels}px - {center_coord[0]:.3f}, {center_coord[1]:.3f} - {START_DATE} - {END_DATE}.png')
plt.show()